# <font color='red'> Import libraries </font>

In [1]:
# work with dataframes
import pandas as pd
import numpy as np

# we'll use this to store the a dataframe to csv at a later stage
import csv

# make HTTP requests to a specified URL
import requests

# web scraping library
from bs4 import BeautifulSoup

# time management
import time
from random import randint
# from collections import Counter

# regular expressions
import re

# io operations 
import os

# read  and write img links
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# <font color='red'> Download Images <font/>

In [ ]:
### create dict of postID , images to download

# get image links
img_urls = []

# loop through the city pages
for url in list_of_cities:
    # instantiate a BS object
    soup = BeautifulSoup(requests.get(url).text, 'lxml')

    # img with dimensions place holder
    # img_url = 'https://images.craigslist.org/{}_1200x900.jpg' <--- for high resolution images
    img_url = 'https://images.craigslist.org/{}_300x300.jpg'
    for i, (a_img, a_title) in enumerate(zip(soup.select('.result-row a.result-image.gallery[data-ids]'),
                              soup.select('.result-row a.result-title')), 1):
        # create a list of images for each vehicle
        img_links = []
        
        # create a dictionary to store (postID, imageLinks)        
        d = {}
        
        # default dict setup
        d.setdefault(a_title['id'],[])
        
        # for each postID get img links
        for data_id in [s.split(':')[1] for s in a_img['data-ids'].split(',')]:
            # store img links
            img_links.append(img_url.format(data_id))
        
        # append postID and list of img links
        d[a_title['id']].append(img_links)
        img_urls.append(d)        
        time.sleep(randint(0,1))

In [ ]:
# save image dicts in json
try:
    with open('craigslistproject/c_img_urls.json', 'w', encoding='utf-8') as f:
        json.dump(img_urls, f, ensure_ascii=False, indent=4)
except IOError:
    print("I/O error")

In [ ]:
# open json list of dicts
with open('craigslistproject/c_img_urls.json') as f:
  img_paths = json.load(f)

In [ ]:
# helper method to create directories in local
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
# iterate through list of dicts
for car in img_paths:
    # for each postID get links
    for post_id, photo_links in car.items():
        # image name increment
        img_number = 0
        # index through list of img urls
        for link in photo_links[0]:
            Picture_request = requests.get(link)
            # create folder name matching postID
            createFolder('craigslist_imgs\\' + post_id)
            # check if img is present
            if Picture_request.status_code == 200:
                # if present create img name
                f_name = post_id + '_{}.jpg'.format(img_number)
                # create image path for storage
                f_path = r'c:\datascience\craigslist_imgs\{}{}{}'.format(post_id,'\\', f_name)
                # store image in created directory
                with open(f_path, 'wb') as f:
                    for chunk in Picture_request.iter_content(chunk_size=1024):
                        length = f.write(chunk)
                img_number +=1

## <font color='red'> Redownload Failed Images <font/>

In [ ]:
# # iterate through list of downloaded folders
# test = img_paths
# count = 1
# remaining = []

# # get the names f downloaded postIDs
# for car in test:
#     # for each postID get links 9 (list if dicts)
#     for post_id, photo_links in car.items():
#         if post_id not in remove_folders:
#             remaining.append(car)
    
# len(test), len(remaining), len(remove_folders)


# # reinitiate downloads
# # iterate through list of dicts
# for car in remaining:
#     # for each postID get links 9 (list if dicts)
#     for post_id, photo_links in car.items():
#         # image name incrementer
#         img_number = 0
#         # index through list of img urls in dict item
#         for link in photo_links[0]:
#             Picture_request = requests.get(link)
#             # create folder with postID name
#             createFolder('B:\craigslist_imgs\\' + post_id)
#             # check if img is present
#             if Picture_request.status_code == 200:
#                 # if present create img name
#                 f_name = post_id + '_{}.jpg'.format(img_number)
#                 # create image path for local storage
#                 f_path = r'B:\craigslist_imgs\{}{}{}'.format(post_id,'\\', f_name)
#                 # store image in created directory
#                 with open(f_path, 'wb') as f:
#                     for chunk in Picture_request.iter_content(chunk_size=1024):
#                         length = f.write(chunk)
#                 img_number +=1
#                 time.sleep(randint(0,1))